In [94]:
!pip install hmmlearn==0.2.7

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [95]:
import hmmlearn
from hmmlearn import hmm
import numpy as np
import random
import itertools
import math

In [96]:
def guitar_frets_name():
    # 開放弦から12フレット分
    frets = [
        ["E5", "F5", ["F#5", "G-5"], "G5", ["G#5", "A-5"], "A5", ["A#5", "B-5"], "B5", "C6", ["C#6", "D-6"], "D6", ["D#6", "E-6"], "E6"],
        ["B4", "C5", ["C#5", "D-5"], "D5", ["D#5", "E-5"], "E5", "F5", ["F#5", "G-5"], "G5", ["G#5", "A-5"], "A5", ["A#5", "B-5"], "B5"],
        ["G4", ["G#4", "A-4"], "A4", ["A#4", "B-4"], "B4", "C5", ["C#5", "D-5"], "D5", ["D#5", "E-5"], "E5", "F5", ["F#5", "G-5"], "G5"],
        ["D4", ["D#4", "E-4"], "E4", "F4", ["F#4", "G-4"], "G4", ["G#4", "A-4"], "A4", ["A#4", "B-4"], "B4", "C5", ["C#5", "D-5"], "D5"],
        ["A3", ["A#3", "B-3"], "B3", "C4", ["C#4", "D-4"], "D4", ["D#4", "E-4"], "E4", "F4", ["F#4", "G-4"], "G4", ["G#4", "A-4"], "A4"],
        ["E3", "F3", ["F#3", "G-3"], "G3", ["G#3", "A-3"], "A3", ["A#3", "B-3"], "B3", "C4", ["C#4", "D-4"], "D4", ["D#4", "E-4"], "E4"]
    ]
    return frets

In [97]:
def guitar_frets_dict():
    # 開放弦から20フレット分
    frets = [
        {"E5": 0, "F5": 6,  "F#5":12, "G5":18, "G#5":24, "A5":30, "A#5":36, "B5": 42, "C6": 48, "C#6": 54, "D6": 60, "D#6": 66,"E6": 72,"F6":78,"F#6":84,"G6":90,"G#6":96,"A6":102,"A#6":108,"B6":114,"C7":120},
        {"B4": 1, "C5": 7,  "C#5":13, "D5":19, "D#5":25, "E5":31, "F5": 37, "F#5":43, "G5": 49, "G#5": 55, "A5": 61, "A#5": 67,"B5": 73,"C6":79,"C#6":85,"D6":91,"D#6":97,"E6":103,"F6":109,"F#6":115,"G6":121},
        {"G4": 2, "G#4":8,  "A4": 14, "A#4":20,"B4": 26, "C5":32, "C#5":38, "D5": 44, "D#5":50, "E5":  56, "F5": 62, "F#5": 68,"G5": 74,"G#5":80,"A5":86, "A#5":92,"B5":98,"C6":104,"C#6":110,"D6":116,"D#6":122},
        {"D4": 3, "D#4":9,  "E4": 15, "F4":21, "F#4":27, "G4":33, "G#4":39, "A4": 45, "A#4":51, "B4":  57, "C5": 63, "C#5": 69,"D5": 75,"D#5":81,"E5":87, "F5":93,"F#5":99,"G5":105,"G#5":111,"A5":117,"A#5":123},
        {"A3": 4, "A#3":10, "B3": 16, "C4":22, "C#4":28, "D4":34, "D#4":40, "E4": 46, "F4": 52, "F#4": 58, "G4": 64, "G#4": 70,"A4": 76,"A#4":82,"B4":88, "C5":94,"C#5":100,"D5":106,"D#5":112,"E5":118,"F5":124},
        {"E3": 5, "F3": 11, "F#3":17, "G3":23, "G#3":29, "A3":35, "A#3":41, "B3": 47, "C4": 53, "C#4": 59, "D4": 65, "D#4": 71,"E4": 77,"F4":83,"F#4":89, "G4":95,"G#4":101,"A4":107,"A#4":113,"B4":119,"C5":125}
    ]
    note_number = [i for i in range(0, 126)]
    return note_number, frets

In [98]:
def cal_dist(a,b):
  dist = abs(math.floor(a / 6) - math.floor(b / 6)) #次の音へのフレットの距離
  return dist

In [99]:
def make_dist_list():
  dist_list = [] #次の音まで何フレット離れているかを格納するリスト

  for i in range(len(observes[0])-1): #最後の要素は次の音がないから走査する必要ない
    dist = cal_dist(observes[0][i],observes[0][i+1])
    dist_list.append(dist)

  return dist_list

In [100]:
#正規化する関数

def normalization(data,sum_list):
  for i in range(n_observe_states):
    for j in range(n_observe_states):
      data[i][j] = data[i][j]/sum_list[i]

  return data

In [101]:
#初期確率

def init_startprob():
  start = []
  for i in range(n_observe_states):
    start.append(1/n_observe_states)

  start = np.array(start)
  return start
  

In [102]:
#出力確率

def init_emmisionprob():
  emmision = []

  sum_list = [0]*n_observe_states #各行の数値の合計値のリスト

  #100,80,1の割り当て
  for i in range(n_observe_states):
    i_key = frets_items[i][0]
    for j in range(n_observe_states):
      if i == j:#自分のキーと要素が同じなら100
        emmision.append(100)
        sum_list[i] += 100

      elif frets_items[j][0] == i_key:#同じ音なら80
        emmision.append(80)
        sum_list[i] += 80

      else:
        emmision.append(1)
        sum_list[i] += 1
    
  #126*126の行列にする
  emmision = np.array(emmision).reshape(n_observe_states,n_observe_states)

  #numpy配列はint型だからfloatにする
  emmision = emmision.astype(np.float64)

  #正規化
  emmision = normalization(emmision,sum_list)

  
  return emmision


In [103]:
def get_items():
  items = []

  #全部のキーと要素を抽出
  for i in range(6):
    items.append(list(frets[i].items()))

  # print(items)

  #二次元配列になっているので平坦化する
  items = list(itertools.chain.from_iterable(items))

  #番号順にソートする
  items = sorted(items, key = lambda x:x[1])

  return items

In [104]:
#(S,F)としたときにS+F = xとする． S = 弦，F = フレット ，0から125まで

observe_states, frets = guitar_frets_dict()
n_observe_states = len(observe_states)

# print(frets[1].keys())  #キーのみ出力
# print(frets[1].values())  #値のみの出力
# E4 = [d.get('E4') for d in frets]#E4の数字取得
# print(E4)



In [105]:
observes = [[22,46,33]] #22,33の順番の観測
n_observes = len(observes)



print(n_observe_states)

126


In [106]:
# 初期状態確率，全部同じ
startprob = init_startprob()

startprob

array([0.00793651, 0.00793651, 0.00793651, 0.00793651, 0.00793651,
       0.00793651, 0.00793651, 0.00793651, 0.00793651, 0.00793651,
       0.00793651, 0.00793651, 0.00793651, 0.00793651, 0.00793651,
       0.00793651, 0.00793651, 0.00793651, 0.00793651, 0.00793651,
       0.00793651, 0.00793651, 0.00793651, 0.00793651, 0.00793651,
       0.00793651, 0.00793651, 0.00793651, 0.00793651, 0.00793651,
       0.00793651, 0.00793651, 0.00793651, 0.00793651, 0.00793651,
       0.00793651, 0.00793651, 0.00793651, 0.00793651, 0.00793651,
       0.00793651, 0.00793651, 0.00793651, 0.00793651, 0.00793651,
       0.00793651, 0.00793651, 0.00793651, 0.00793651, 0.00793651,
       0.00793651, 0.00793651, 0.00793651, 0.00793651, 0.00793651,
       0.00793651, 0.00793651, 0.00793651, 0.00793651, 0.00793651,
       0.00793651, 0.00793651, 0.00793651, 0.00793651, 0.00793651,
       0.00793651, 0.00793651, 0.00793651, 0.00793651, 0.00793651,
       0.00793651, 0.00793651, 0.00793651, 0.00793651, 0.00793

In [107]:
frets_items = get_items()


frets_items[3][0]
# name_list[1]

'D4'

In [108]:
# 出力確率

emmisionprob = init_emmisionprob()

print(emmisionprob[0])

# emmisionprob[0][0]




[0.18484288 0.00184843 0.00184843 0.00184843 0.00184843 0.00184843
 0.00184843 0.00184843 0.00184843 0.00184843 0.00184843 0.00184843
 0.00184843 0.00184843 0.00184843 0.00184843 0.00184843 0.00184843
 0.00184843 0.00184843 0.00184843 0.00184843 0.00184843 0.00184843
 0.00184843 0.00184843 0.00184843 0.00184843 0.00184843 0.00184843
 0.00184843 0.14787431 0.00184843 0.00184843 0.00184843 0.00184843
 0.00184843 0.00184843 0.00184843 0.00184843 0.00184843 0.00184843
 0.00184843 0.00184843 0.00184843 0.00184843 0.00184843 0.00184843
 0.00184843 0.00184843 0.00184843 0.00184843 0.00184843 0.00184843
 0.00184843 0.00184843 0.14787431 0.00184843 0.00184843 0.00184843
 0.00184843 0.00184843 0.00184843 0.00184843 0.00184843 0.00184843
 0.00184843 0.00184843 0.00184843 0.00184843 0.00184843 0.00184843
 0.00184843 0.00184843 0.00184843 0.00184843 0.00184843 0.00184843
 0.00184843 0.00184843 0.00184843 0.00184843 0.00184843 0.00184843
 0.00184843 0.00184843 0.00184843 0.14787431 0.00184843 0.0018

In [109]:
frets_dist = make_dist_list()

print(frets_dist)

[4, 2]


In [110]:
# 遷移確率 
transmat = []

sum_list = [0]*n_observe_states #各行の数値の合計値のリスト


#とりあえず50を割り当て
for i in range(n_observe_states):
  for j in range(n_observe_states):
   transmat.append(50)
   sum_list[i] += 50


transmat = np.array(transmat).reshape(n_observe_states,n_observe_states) #126*126の行列にする

if frets_dist:
  for i in range(len(frets_dist)):

    #4フレット離れていたら1を割り当て
    if frets_dist[i] >= 4:
      observe_states_index1 = observes[0][i]  #現在の音
      observe_states_index2 = observes[0][i+1] #次の音

      transmat[observe_states_index1][observe_states_index2] = 1
      sum_list[observe_states_index1] -= 49 #合計値が変わるので49引く

      key = frets_items[observe_states_index2][0] #keyの代入
      print(key)

      #同じ音かつフレットの距離が2以下の時80を割り当て
      for j in range(n_observe_states):
        dist = cal_dist(observe_states_index1,j)
        
        if key == frets_items[j][0] and dist <= 2: 
          print("dist=",dist)
          transmat[observe_states_index1][j] = 80
          sum_list[observe_states_index1] += 30
  



print(transmat[22])

#numpy配列はint型だからfloatにする
transmat = transmat.astype(np.float64)


#正規化

transmat = normalization(transmat,sum_list)


print(transmat[22])


E4
dist= 1
[50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 80 50 50 50 50 50 50 50 50
 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50  1 50
 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50
 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50
 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50 50
 50 50 50 50 50 50]
[0.00796052 0.00796052 0.00796052 0.00796052 0.00796052 0.00796052
 0.00796052 0.00796052 0.00796052 0.00796052 0.00796052 0.00796052
 0.00796052 0.00796052 0.00796052 0.01273683 0.00796052 0.00796052
 0.00796052 0.00796052 0.00796052 0.00796052 0.00796052 0.00796052
 0.00796052 0.00796052 0.00796052 0.00796052 0.00796052 0.00796052
 0.00796052 0.00796052 0.00796052 0.00796052 0.00796052 0.00796052
 0.00796052 0.00796052 0.00796052 0.00796052 0.00796052 0.00796052
 0.00796052 0.00796052 0.00796052 0.00796052 0.00015921 0.00796052
 0.00796052 0.00796052 0.00796052 0.00796052 0.00796052 0.00796052
 

In [111]:
# hmmlearn のインスタンス作成
model = hmm.MultinomialHMM(n_components = n_observe_states)

model.n_features = n_observe_states #状態の数
model.startprob_= startprob #初期確率
model.transmat_= transmat #遷移確率
model.emissionprob_= emmisionprob #出力確率



In [112]:
order_node = np.array(observes).T

print(order_node)

# 推定
logprob, decoded_codes = model.decode(order_node)
print(decoded_codes)
print("type=",type(decoded_codes))

for i in range(len(decoded_codes)):
  print(math.floor(decoded_codes[i] % 6)+1,"弦 ",math.floor(decoded_codes[i]/6),"フレット")

# print("観測:")
# for i in range(len(order_node)):
#   print()



[[22]
 [46]
 [33]]
[22 15 33]
type= <class 'numpy.ndarray'>
5 弦  3 フレット
4 弦  2 フレット
4 弦  5 フレット
